# Installation / Importation

In [2]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from office365.sharepoint.listitems.caml.query import CamlQuery

import pandas as pd
import numpy as np
import docx
import io

from datetime import datetime
from dateutil.parser import parse
import time

from docx.shared import Cm, Inches, Pt
from docx.shared import RGBColor
from docx.enum.section import WD_SECTION
from docx.enum.table import WD_TABLE_ALIGNMENT, WD_ALIGN_VERTICAL
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.style import WD_STYLE_TYPE
from docx.oxml.ns import nsdecls
from docx.oxml import parse_xml
#from docx.oxml.shared import qn

from PIL import Image, ImageOps

# Acces API Sharepoint

In [3]:
## Définir des fonctions pour récupérer et sauvegarder les données de Sharepoint

def get_sharepoint_list(ctx, listname, filter_query = None):
    """
    Fonction de récupération des éléments de la liste SharePoint via le connecteur au site
    Avec possibilité de filtrer la liste via un critère défini
    Parameters
    ----------
    listname : nom de la liste sharepoint à récupérer (string)
    filter_query : Caml Query to filter the data (string)

    Returns
    -------
    items : Objet contenant toutes les lignes de la liste Sharepoint.

    """
    caml_query = CamlQuery.parse(filter_query)
    sp_list = ctx.web.lists.get_by_title(listname)
    items = sp_list.get_items(caml_query)
    ctx.load(sp_list)
    ctx.execute_query()
    
    return items

def save_attachment_to_list(ctx, item):
    """
    Fonction de récupérer des pièces jointes en binaire

    Parameters
    ----------
    item : élément de la liste SharePoint (item object)
    
    Returns
    -------
    liste de noms et de contenus binaires des pièces jointes.

    """
    dico = {}
    
    if item.properties['Attachments']: 
        attachment_files = item.attachment_files.get().execute_query()
        list_attachment = []
        for att in attachment_files:
            response = File.open_binary(ctx, att.server_relative_url)
            img = io.BytesIO(response.content) 
            list_attachment.append({"Nom" : att.file_name, "Contenu" : img})
        dico[item.id] = list_attachment
    
    return list(dico.values())

def spList_todf(ctx, list_name, filter_query=None):
    """
    Fonction de transformation de la liste SharePoint en DataFrame
    
    Parameters
    ----------
    items : Liste SharePoint préalablement chargée 

    Returns
    -------
    Tableau pandas DataFrame contenant toutes 
    les lignes de la liste Sharepoint.

    """
    items = get_sharepoint_list(ctx, list_name, filter_query)
    notcols = ['ID',
        'FileSystemObjectType', 'ServerRedirectedEmbedUri',
        'ServerRedirectedEmbedUrl', 'ContentTypeId', 'AuthorId', 'EditorId',
        'OData__UIVersionString', 'GUID', 'ComplianceAssetId'
    ]

    df = pd.DataFrame(items.to_json()).drop(notcols, axis=1)

    list_att = []
    for i in range(len(df)):
        list_att.append(sum(save_attachment_to_list(ctx, items[i]), []))

    if len(sum(list_att, [])) > 0:
        df['PJ'] = list_att

    return df

In [4]:
client_id = '74de3bbf-590f-4d6e-8325-376841457ce4'                  # Evere
client_secret = 'czmVcHrs9txSVdlzg39BL+sjh0ezxBdpQlalBcvTOJ8='      # Evere
site_url = 'https://naldeo.sharepoint.com/sites/MonitoringEveR/BDD' # Evere

In [5]:
## Définir les modalités de la lecture et l'écriture de rapports
# Adresse du modèle : base_url\{template}.docx
# Adresse du fichier export : base_url\{export}.docx

c = 0
# Nom du modèle 
template = 'Rapport_Evere_Modele'

# Nom du fichier export
export = 'Export_test_{}'

# Base uri One Drive 
base_url = r'C:\Users\minh.nguyen\OneDrive - Naldeo\Documents\2022_Projet_TerraMonitoring\{}.docx'

# Définir l'id_visite
id_visite = 192

In [6]:
# Requête filtrant
caml_query_text = '<Where><Eq><FieldRef Name="IDvisite" /><Value Type="Integer">{}</Value></Eq></Where>'

# Connecter avec les références SharePoint Online using App Only
context_auth = AuthenticationContext(url=site_url)
context_auth.acquire_token_for_app(client_id, client_secret)

ctx = ClientContext(site_url, context_auth)
web = ctx.web
ctx.load(web)
ctx.execute_query()

In [7]:
## Transformer Sharepoint en DataFrame
# Liste Commentaires sans filtre + sans PJ
items = get_sharepoint_list(ctx, 'Commentaires')
notcols = ['ID',
        'FileSystemObjectType', 'ServerRedirectedEmbedUri',
        'ServerRedirectedEmbedUrl', 'ContentTypeId', 'AuthorId', 'EditorId',
        'OData__UIVersionString', 'GUID', 'ComplianceAssetId'
    ]
df = pd.DataFrame(items.to_json()).drop(notcols, axis=1)

# Créer une colonne Date de détection qui garde en mémoire la première date de détection
df['date_detection'] = df.groupby('ref_id_unique')['Created'].transform('min')

In [8]:
# Liste Commentaires avec filtre
st = time.time()
df_commentaires = spList_todf(ctx, 'Commentaires', caml_query_text.format(id_visite))
et = time.time()
print(et-st)

209.27667427062988


In [9]:
df_commentaires = pd.merge(df, df_commentaires, on=[*list(df_commentaires.columns.drop('PJ'))], how='right')
# Remplacement des valeurs None des colonnes pour éviter l'erreur NoneType
# [id_sous_unite_N2] --> type float
df_commentaires.id_sous_unite_N2.fillna(value=np.nan, inplace=True)

In [10]:
# Récupération des données d'autres listes
df_unites = spList_todf(ctx, 'Unites')
df_sous_unitesn1 = spList_todf(ctx, 'Sous_unites_niveau1')
df_sous_unitesn2 = spList_todf(ctx, 'Sous_unites_niveau2')

#4 Jointure des 4 tableaux
df_sous_unitesn1.rename(columns={'dernier_niveau': 'dernier_niveau_su1'}, inplace=True)
df_unites.rename(columns={'dernier_niveau': 'dernier_niveau_u'}, inplace=True)

df_merged = df_commentaires.merge(df_sous_unitesn2[['Id', 'Title']],
                                  how='left',
                                  left_on='id_sous_unite_N2',
                                  right_on='Id',
                                  suffixes=('','_su2'))\
                           .merge(df_sous_unitesn1[['Id', 'Title', 'dernier_niveau_su1']],
                                  how='left',
                                  left_on='id_sous_unite_N1',
                                  right_on='Id',
                                  suffixes=('', '_su1'))\
                           .merge(df_unites[['Id', 'Title', 'dernier_niveau_u']],
                                  how = 'left',
                                  left_on = 'id_unite',
                                  right_on = 'Id',
                                  suffixes = ('','_u'))

# Export Word

In [11]:
### FONCTION ###
def text_style_in_table(txt): 
    """
    Mettre en forme le texte dans un tableau
    txt : cellule d'un tableau
    """
    # Régler la taille de police
    txt.paragraphs[0].runs[0].font.size = Pt(8)
    
    # Régler l'espace avant 
    txt.paragraphs[0].paragraph_format.space_before = Pt(3)
    
    # Régler l'espace après
    txt.paragraphs[0].paragraph_format.space_after = Pt(3)
    
    # Régler l'alignement à gauche
    txt.paragraphs[0].alignment = WD_ALIGN_PARAGRAPH.LEFT

In [12]:
def resize(image, px):
    """
    Redimensionner le containeur (carré) d'une image 
    L'image sera redimensionnée proportionnellement suivant la taille du containeur
    HD (cm) --> pixel / 60 
    L'image sera reconvertie en bytes 
    ----
    image : image
    px : taille en pixel
    ----
    objet byte de l'image redimensionnée
    """
    img = Image.open(image)
    byteIO = io.BytesIO()
    if img.mode == 'CMYK':
        ImageOps.contain(img, (px, px)).convert('RGB').save(byteIO, format=img.format)
    else :
        ImageOps.contain(img, (px, px)).save(byteIO, format=img.format)
    return byteIO

## Modèle V2

In [13]:
## MODELE V1 // Définition des couleurs des états
colors = {'Détecté': 'FF7F00',      #orange
 'Perdure et pas agravé': '6495ED', #lightblue
 'Perdure et agravé': 'F88379',     #coral
 'Réparation provisoire': 'FFD662', #gold 
 'Réglé': '3EB489',                 #mintgreen
 'Stand-by': 'D3D3D3'}              #lightgray

## MODELE V2 // Définition des couleurs des criticités
colors_crit = {
    'Immédiat' : 'FF0000',        #red
    'Prioritaire' : 'FF7F00',     #orange
    'Secondaire' : '000000'}      #black

In [14]:
# Transformer la colonne Etat en catégorielle pour la rendre ordonnée
df_merged.Etat = df_merged.Etat.astype('category')
df_merged.Etat = df_merged.Etat.cat.set_categories(list(colors.keys()))

### Synthèse

In [ ]:
#######################################################################
###########                     SYNTHESE                    ###########
#######################################################################

In [ ]:
test = df_merged.copy()
test['Loc'] =

In [15]:
### FONCTION ###
def write_in_table(dataframe):
    """
    Sélectionner des données à partir d'un dataframe et les mettre en tableau synthèse (sans photo)
    """
    # Filtrer seulement la visite sélectionnée
    df = dataframe.copy()
    #df = df[df.IDvisite == id_visite]
    
    # Combiner la localisation de Unité, Sous-unité N1 et N2 dans une nouvelle colonne
    df['Localisation'] = (df[['Id','Title_su1', 'Title_su2']].set_index('Id')
                                                                        .stack()
                                                                        .groupby(level=0, sort=False)
                                                                        .agg(' - '.join)
                                                                        .values)
    # Combiner le titre et le commentaire dans une nouvelle colonne
    df['Observation'] = (df[['Id','Title', 'Commentaire']].set_index('Id')
                                                          .stack()
                                                          .groupby(level=0, sort=False)
                                                          .agg(' : '.join)
                                                          .values)
    
    # Combiner l'action et la durée 
    # (!) plus lent sur un grand jeu de données mais accepte les 2 valeurs Null 
    df['Action_Duree'] = df[['Action', 'Duree']].apply(lambda x: ' sous '.join(x.dropna()), axis=1)
    
    # Combiner la criticité et le domaine 
    # Il faut au moins une valeur notNull
    #df['Criticite_Domaine'] = (df[['Id','Criticite', 'Domaine']].set_index('Id').stack().groupby(level=0, sort=False).agg(' - '.join).values)
    df['Criticite_Domaine'] = df[['Criticite', 'Domaine']].apply(lambda x: ' - '.join(x.dropna()), axis=1)
    
    # Sélectionner les colonnes nécessaires
    df = df[['Id',     
             'Localisation',
             'Observation',         
             'date_detection', 
             'Etat', 
             'Action_Duree',
             'Criticite_Domaine',
             'Criticite',
             'Domaine',
             'Title_u',
             'id_sous_unite_N1', 
             'id_sous_unite_N2']].reset_index(drop=True) 
    
    # Convertir la colonne de date 
    df.date_detection = pd.to_datetime(df.date_detection).dt.strftime("%d/%m/%Y")
    
    # Classer dans l'ordre de la sous unite N1, N2 et l'état (priorité localisation)
    # df = df.sort_values(by=['id_sous_unite_N1', 'id_sous_unite_N2','Etat']).reset_index(drop=True)
    
    # Classer dans l'ordre de la sous unite N1, N2 et l'état (priorité etat)
    df = df.sort_values(by=['Etat', 'id_sous_unite_N1', 'id_sous_unite_N2']).reset_index(drop=True)
    
    # Colonnes de la table 
    syn_col = ["Numéro d’observation", 
               "Localisation", 
               "Commentaire", 
               "Détecté le", 
               "Etat", 
               "Action à mener + durée si complétée", 
               "Criticité + domaine"]
        
    # Initialiser un tableau 
    table = doc.add_table(rows=1, cols=len(syn_col), style="Table Grid")
    # Régler l'alignement
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    table.allow_autofit = True 
    # Fixer la taille de la 2eme colonne  
    table.columns[2].width = Cm(8) 
    # Fixer la taille de deux dernières colonnes 
    table.columns[len(syn_col)-2].width = Cm(4) 
    table.columns[len(syn_col)-1].width = Cm(4)

    # Créer l'entête
    hdr_cells = table.rows[0].cells
    for i in range(len(syn_col)): 
        hdr_cells[i].text = syn_col[i]

    # Ajouter des données
    for i in range(len(df)):
        color = colors.get(df.Etat[i])
        color_crit = colors_crit.get(df.Criticite[i])
        
        # Pour chaque observation, ajouter une ligne
        row_cells = table.add_row().cells
        
        # Remplir le tableau avec les données du dataframe
        for j in range(len(syn_col)): 
            row_cells[j].text = str(df.iloc[i,j])
            
            # Mettre l'état (5e colonne) en gras et avec les couleurs correspondantes
            if j == 4: 
                for paragraph in row_cells[j].paragraphs:
                    for run in paragraph.runs:
                        run.font.bold = True
                        run.font.color.rgb = RGBColor.from_string(color)
    
    # Appliquer le style de la police
    for column in table.columns :
        for cell in column.cells : 
            for paragraph in cell.paragraphs :
                if len(paragraph.runs) > 0:
                    text_style_in_table(cell)     

In [17]:
### EXECUTION ###
# Initialiser un document à partir du template
doc = docx.Document(base_url.format(template))

# Sélectionner la police
font = doc.styles['Normal'].font
font.name = 'Arial'

# Parcourir la liste les unités 
for u in df_merged.sort_values(by='id_unite').Title_u.dropna().unique():
    
    # Créer 1 dataframe pour chaque unité 
    df_us = df_merged.groupby('Title_u').get_group(u).reset_index(drop=True)
    
    # Ajouter un titre 
    doc.add_heading(u, 1)
    
    try :
        #section = doc.sections[-1]
        #section.start_style = WD_SECTION.CONTINUOUS
        #section.left_margin = Cm(0.5)        
        write_in_table(df_us)
    except : 
        pass
    
    
    
# Exporter le fichier doc au même endroit
doc.save(base_url.format(export.format(str(c))))
c = c+1

### Photos

In [ ]:
#######################################################################
###########                      PHOTOS                     ###########
#######################################################################

In [48]:
### FONCTION ###
def write_each(df):
    """
    Ecrire un tableau en détails pour chaque commentaire 
    """
    # Filtrer seulement la visite sélectionnée
    # df = df[df.IDvisite == id_visite]
    
    # Classer dans l'ordre de la sous unite N1, N2 et l'état (priorité etat)
    df = df.sort_values(by=['Etat', 'id_sous_unite_N1', 'id_sous_unite_N2']).reset_index(drop=True)
    
    for i in range(len(df)):
        color = colors.get(df.Etat[i])
        color_crit = colors_crit.get(df.Criticite[i])
        shading_elm_1 = parse_xml(r"<w:shd {} w:fill='{}'/>".format(nsdecls('w'), '6699CC')) #color

        ## TEXT ##
        table = doc.add_table(rows=1, cols=4, style="Table Grid")

        # ID du commentaire
        cell_id = table.rows[0].cells
        cell_id[0].text = str(df.Id[i])
        cell_id[0]._tc.get_or_add_tcPr().append(shading_elm_1)
        cell_id[1].merge(cell_id[3])
        cell_id[1].text = ' '
        
        # Localisation
        cell_loc = table.add_row().cells
        cell_loc[0].text = 'Localisation :' 
        cell_loc[1].merge(cell_loc[3])
        if df.dernier_niveau_u[i]:
            cell_loc[1].text = df.Title_u[i]
        else : 
            if df.dernier_niveau_su1[i]:   
                cell_loc[1].text = df.Title_su1[i]
            else:
                cell_loc[1].text = str(df.Title_su1[i]) + ' - ' + str(df.Title_su2[i])

        # Titre + Commentaire
        cell_com = table.add_row().cells
        cell_com[0].text = 'Commentaire :'
        cell_com[1].merge(cell_com[3])
        if df.Commentaire[i] is None :
            cell_com[1].text = df.Title[i] 
        else : 
            cell_com[1].text = df.Title[i] + ' : ' + df.Commentaire[i]


        # Date de détection
        cell_date = table.add_row().cells
        cell_date[0].text = 'Détecté le :'
        cell_date[1].merge(cell_date[3])
        cell_date[1].text = pd.to_datetime(df.date_detection[i]).strftime("%d/%m/%Y")


        # Etat + Action
        cell_etact = table.add_row().cells
        cell_etact[0].text = 'Etat :'
        cell_etact[1].text = df.Etat[i]
        cell_etact[2].text = 'Action à mener :'
        if df.Action[i] is None :
            cell_etact[3].text = ' '
        else : 
            if df.Duree[i] is None :
                cell_etact[3].text = df.Action[i]
            else : 
                cell_etact[3].text = df.Action[i] + ' sous ' + df.Duree[i]

         # Criticité + Domaine
        cell_crit = table.add_row().cells
        cell_crit[0].text = 'Criticité :' 
        if df.Criticite[i] is None :
            cell_crit[1].text = ' '
        else :
            cell_crit[1].text = df.Criticite[i]
            cell_crit[1].paragraphs[0].runs[0].font.bold = True
            cell_crit[1].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string(color_crit)
        cell_crit[2].text = 'Domaine :'
        if df.Domaine[i] is None :
            cell_crit[3].text = ' '
        else :
            cell_crit[3].text = df.Domaine[i]

        # Mettre en forme la police
        for column in table.columns :
            for cell in column.cells : 
                for paragraph in cell.paragraphs :
                    if len(paragraph.runs) > 0:
                        text_style_in_table(cell)    

        # Mises en forme particulières
        cell_id[0].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string('FFFFFF')
        cell_id[0].paragraphs[0].runs[0].font.size = Pt(10)
        cell_id[0].paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        cell_etact[1].paragraphs[0].runs[0].font.bold = True
        cell_etact[1].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string(color)

        ## PHOTOS ##
        # définir le nombre de colonnes du tableau photos
        nbcol = 3
        if len(df.PJ[i]) > 0:
            # ajouter une ligne au tableau précédent
            cell_photo = table.add_row().cells

            # fusionner toutes les colonnes
            cell_photo[0].merge(cell_photo[3])

            # déterminer le nombre de lignes du tableau photos
            if len(df.PJ[i]) % nbcol == 0:
                cntrows = len(df.PJ[i]) // nbcol
            else:
                cntrows = len(df.PJ[i]) // nbcol + 1

            # ajouter le tableau photos dans la cellule fusionnée
            tbl_photo = cell_photo[0].add_table(rows=cntrows, cols=3) 
            tbl_photo.allow_autofit = True 

            # parcourir chaque ligne et colonne du tableau photos 
            # ajouter les photos au fur à mesure
            for row in range(cntrows):
                for col in range(nbcol):
                    paragraph = tbl_photo.rows[row].cells[col].paragraphs[0]
                    run = paragraph.add_run()
                    try : 
                        img = Image.open(df.PJ[i][row*nbcol+col]['Contenu'])
                        # selon l'orientation de la photo, ajuster la taille 
                        if img.width > img.height:
                            run.add_picture(resize(df.PJ[i][row*nbcol+col]['Contenu'], 250),
                                        width=Cm(5))
                        else:
                            run.add_picture(resize(df.PJ[i][row*nbcol+col]['Contenu'], 250),
                                        height=Cm(4))
                    except :
                        pass  

In [49]:
### EXECUTION ###
doc = docx.Document(base_url.format(template))

# Sélectionner la police
font = doc.styles['Normal'].font
font.name = 'Arial'

# Parcourir la liste les unités 
for u in df_merged.sort_values(by='id_unite').Title_u.dropna().unique():
    
    # Créer 1 dataframe pour chaque unité 
    df_us = df_merged.groupby('Title_u').get_group(u).reset_index(drop=True)
    
    # Ajouter un titre 
    doc.add_heading(u, 1)
    
    try :
        write_each(df_us)
        
    except : 
        pass

# Exporter le fichier doc au même endroit
doc.save(base_url.format(export.format(str(c))))
c = c+1

In [50]:
doc = docx.Document()

for u in df_merged.sort_values(by='id_unite').Title_u.dropna().unique():
    
    # Créer 1 dataframe pour chaque unité 
    df_us = df_merged.groupby('Title_u').get_group(u).reset_index(drop=True) 
    df = df_us.sort_values(by=['Etat', 'id_sous_unite_N1', 'id_sous_unite_N2']).reset_index(drop=True)
    
    doc.add_heading(u, 1)
    for i in range(len(df)): 
        color = colors.get(df.Etat[i])
        color_crit = colors_crit.get(df.Criticite[i])
        shading_elm_1 = parse_xml(r"<w:shd {} w:fill='{}'/>".format(nsdecls('w'), '6699CC'))  
        table = doc.add_table(rows=1, cols=4, style="Table Grid")
        
       # ID du commentaire
        cell_id = table.rows[0].cells
        cell_id[0].text = str(df.Id[i])
        cell_id[0]._tc.get_or_add_tcPr().append(shading_elm_1)
        cell_id[1].merge(cell_id[3])
        cell_id[1].text = ' '
        
        # Localisation
        cell_loc = table.add_row().cells
        cell_loc[0].text = 'Localisation :' 
        cell_loc[1].merge(cell_loc[3])
        if df.dernier_niveau_u[i]:
            cell_loc[1].text = df.Title_u[i]
        else : 
            if df.dernier_niveau_su1[i]:   
                cell_loc[1].text = df.Title_su1[i]
            else:
                cell_loc[1].text = str(df.Title_su1[i]) + ' - ' + str(df.Title_su2[i])
        
        # Titre + Commentaire
        cell_com = table.add_row().cells
        cell_com[0].text = 'Commentaire :'
        cell_com[1].merge(cell_com[3])
        if df.Commentaire[i] is None :
            cell_com[1].text = df.Title[i] 
        else : 
            cell_com[1].text = df.Title[i] + ' : ' + df.Commentaire[i]

        # Date de détection
        cell_date = table.add_row().cells
        cell_date[0].text = 'Détecté le :'
        cell_date[1].merge(cell_date[3])
        cell_date[1].text = pd.to_datetime(df.date_detection[i]).strftime("%d/%m/%Y")
        
        # Etat + Action
        cell_etact = table.add_row().cells
        cell_etact[0].text = 'Etat :'
        cell_etact[1].text = df.Etat[i]
        cell_etact[2].text = 'Action à mener :'
        if df.Action[i] is None :
            cell_etact[3].text = ' '
        else : 
            if df.Duree[i] is None :
                cell_etact[3].text = df.Action[i]
            else : 
                cell_etact[3].text = df.Action[i] + ' sous ' + df.Duree[i]

         # Criticité + Durée
        cell_crit = table.add_row().cells
        cell_crit[0].text = 'Criticité :' 
        if df.Criticite[i] is None :
            cell_crit[1].text = ' '
        else :
            cell_crit[1].text = df.Criticite[i]
            cell_crit[1].paragraphs[0].runs[0].font.bold = True
            cell_crit[1].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string(color_crit)
        cell_crit[2].text = 'Domaine :'
        if df.Domaine[i] is None :
            cell_crit[3].text = ' '
        else :
            cell_crit[3].text = df.Domaine[i]
            
            
        # Mettre en forme la police
        for column in table.columns :
            for cell in column.cells : 
                for paragraph in cell.paragraphs :
                    if len(paragraph.runs) > 0:
                        text_style_in_table(cell) 
                        
        # Mises en forme particulières
        cell_id[0].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string('FFFFFF')
        cell_id[0].paragraphs[0].runs[0].font.size = Pt(10)
        cell_id[0].paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER
        
        cell_etact[1].paragraphs[0].runs[0].font.bold = True
        cell_etact[1].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string(color)
        
        ## PHOTOS ##
        # définir le nombre de colonnes du tableau photos
        nbcol = 3
        if len(df.PJ[i]) > 0:
            # ajouter une ligne au tableau précédent
            cell_photo = table.add_row().cells

            # fusionner toutes les colonnes
            cell_photo[0].merge(cell_photo[3])

            # déterminer le nombre de lignes du tableau photos
            if len(df.PJ[i]) % nbcol == 0:
                cntrows = len(df.PJ[i]) // nbcol
            else:
                cntrows = len(df.PJ[i]) // nbcol + 1

            # ajouter le tableau photos dans la cellule fusionnée
            tbl_photo = cell_photo[0].add_table(rows=cntrows, cols=3) 

            # parcourir chaque ligne et colonne du tableau photos 
            # ajouter les photos au fur à mesure
            for row in range(cntrows):
                for col in range(nbcol):
                    paragraph = tbl_photo.rows[row].cells[col].paragraphs[0]
                    run = paragraph.add_run()
                    try : 
                        img = Image.open(df.PJ[i][row*nbcol+col]['Contenu'])
                        # selon l'orientation de la photo, ajuster la taille 
                        if img.width > img.height:
                            run.add_picture(resize(df.PJ[i][row*nbcol+col]['Contenu'], 250),
                                        width=Cm(5))
                        else:
                            run.add_picture(resize(df.PJ[i][row*nbcol+col]['Contenu'], 250),
                                        height=Cm(4))
                    except :
                        pass

doc.save('test.docx')

## Modèle V1

### Non formaté

In [ ]:
## MODELE V1 // Définition des couleurs des états
colors = {'Détecté': 'FF7F00',      #orange
 'Perdure et pas agravé': '6495ED', #lightblue
 'Perdure et agravé': 'F88379',     #coral
 'Réparation provisoire': 'FFD662', #gold 
 'Réglé': '3EB489',                 #mintgreen
 'Stand-by': 'D3D3D3'}              #lightgray

# Nombre de colonnes : 
nbcol = 5

In [ ]:
## (Modèle V1) 

## Définition des fonctions pour écrire dans le document Word
def write_table(dataframe, nbcol):
    df = dataframe.copy()
    df = df[df.IDvisite == id_visite].reset_index(drop=True)
    #df = df.sort_values(by=['id_sous_unite_N1', 'id_sous_unite_N2']).reset_index(drop=True)
    
    # Index des lignes du dataframe ayant de pièces-jointes
    atchtrue = [i for i in range(len(df)) if len(df.PJ[i]) > 0]

    # Dataframe des commentaires ayant de pièces-jointes
    df_true = df.loc[atchtrue].explode('PJ').reset_index(drop=True)

    # Le nombre de lignes nécessaire pour afficher les pièces-jointes (5 par ligne)
    if len(df_true.PJ) % nbcol == 0:
        cntrows = len(df_true.PJ) // nbcol
    else:
        cntrows = len(df_true.PJ) // nbcol + 1

    # Initialiser un tableau 
    table = doc.add_table(rows=cntrows*2, cols=nbcol)

    # Remplir le tableau
    for row in range(0, cntrows*2, 2):
        for col in range(nbcol):
            
            ### PHOTO            
            # Initialiser une paragraphe 
            paragraph = table.rows[row].cells[col].paragraphs[0]
            run = paragraph.add_run()

            ## Ajouter une photo dans cette paragraphe
            try:
                img = Image.open(df_true.PJ[row/2*nbcol+col]['Contenu'])
                # Fixer la largeur si la photo est en mode paysage
                if img.width > img.height:
                    run.add_picture(resize(df_true.PJ[row/2*nbcol+col]['Contenu'], 300), width=Cm(3))              
                # Fixer la longeur si la photo est en mode portrait
                else:
                    run.add_picture(resize(df_true.PJ[row/2*nbcol+col]['Contenu'], 300), height=Cm(3))                 
            except:
                pass

            ### TEXTE 
            if len(df_true) > row/2*5+col:
                
                # Index des lignes du même commentaire
                com_gr = list(df_true.groupby('Id').get_group(df_true.Id[row/2*nbcol+col]).index)

                # Déterminer le nombre de lignes et colonnes occupé par chaque commentaire 
                start_row, end_row = (com_gr[0]//nbcol)*2+1, (com_gr[-1]//nbcol)*2+1
                start_col, end_col = com_gr[0]%nbcol, com_gr[-1]%nbcol
                
                # Si c'est sur la même ligne, fusionner les colonnes  
                if start_row == end_row :
                    cell = table.rows[start_row].cells[start_col].merge(table.rows[end_row].cells[end_col])
                
                # Sinon, s'il y a plus de colonnes disponibles à la dernière ligne, utiliser la dernière ligne 
                elif start_col < end_col :
                    cell = table.rows[end_row].cells[0].merge(table.rows[end_row].cells[end_col])
                    
                # Sinon, fusionner les colonnes jusqu'à la fin de la ligne
                else:    
                    cell = table.rows[start_row].cells[start_col].merge(table.rows[start_row].cells[4]) 
                
                # Déterminer la couleur de l'état
                color = colors.get(df_true.Etat[row/2*nbcol+col])

                # Formater la date en format "%d-%m-%Y" : str
                date =  datetime.strftime(parse(df_true.date_detection[row/2*nbcol+col]).date(), "%d-%m-%Y")
                
                try : 
                    df_cell = df_true.drop_duplicates(subset = 'Id')
                # Ajouter le titre et le commentaire
                    if df_cell.Commentaire[row/2*nbcol+col] is not None :
                        cell.add_paragraph(df_cell.Title[row/2*nbcol+col] + ' : ' + df_cell.Commentaire[row/2*nbcol+col]) 
                    else : 
                        cell.add_paragraph(df_cell.Title[row/2*nbcol+col])

                # Ajouter l'état
                    paragraph = cell.add_paragraph()
                    run = paragraph.add_run(df_cell.Etat[row/2*nbcol+col])

                # Changer la couleur de la ligne en couleur de l'état
                    run.font.color.rgb = RGBColor.from_string(color)
                
                 # Ajouter la date
                    paragraph.add_run(' : ' + date)                    
                
                except :
                    pass
                
    # Mettre en forme la police
    for column in table.columns :
        for cell in column.cells : 
            for paragraph in cell.paragraphs :
                if len(paragraph.runs) > 0:
                    text_style_in_table(cell)
                    cell.paragraph[0].runs[0].font.bold = True
                                
def write_no_photo(df):
    # Index des lignes du dataframe sans pièces-jointes (étendu) 1 pièce-jointe/ligne
    atchfalse = [i for i in range(len(df)) if len(df.PJ[i]) == 0]
     
    # Dataframe des commentaires sans pièces-jointes
    df_false = df.loc[atchfalse]
    
    # Parcourir le dataframe
    for i in df_false.index:
        
        # Définir la couleur de l'état & la date
        color = colors.get(df_false.Etat[i])
        date = datetime.strftime(parse(df_false.Created[i]).date(), "%d-%m-%Y")
        
        # Ajouter une paragraphe en liste désordonné
        paragraph = doc.add_paragraph(style = 'List Bullet')
        
        # Ajouter le titre et commentaire
        if df_false.Commentaire[i] is not None : 
            paragraph.add_run(df_false.Title[i] + ' : ' + df_false.Commentaire[i] + '\r')
        else : 
            paragraph.add_run(df_false.Title[i])
        # Ajouter l'état
        run = paragraph.add_run(df_false.Etat[i])
        run.font.color.rgb = RGBColor.from_string(color)
        
        # Ajouter la date
        paragraph.add_run(' : ' + date)  

In [ ]:
## (Modèle V1) Exécuter l'écriture du rapport 

# Initialiser un document à partir du template
doc = docx.Document(base_url.format(template))

# Sélectionner la police
font = doc.styles['Normal'].font
font.name = 'Times News Roman'

# Parcourir la liste les unités 
for u in df_merged.sort_values(by='id_unite').Title_u.dropna().unique():
    
    # Créer 1 dataframe pour chaque unité 
    df_us = df_merged.groupby('Title_u').get_group(u).reset_index(drop=True)
    
    # Ajouter un titre 
    doc.add_heading(u, 1)
 
    # Liste des sous-unités de l'unité active
    sus1 = list(df_us.sort_values(by='id_sous_unite_N1').Title_su1.dropna().unique())
    
    # Parcourir la liste Sous-unités N1 s'elle n'est pas vide
    if len(sus1) > 0 : 
        for su1 in sus1:   
            
            # Créer 1 dataframe pour chaque sous-unité
            df_su1 = df_us.groupby('Title_su1').get_group(su1).reset_index(drop=True)
            
            # Ajouter un titre 
            doc.add_heading(su1, 2)

             # Liste des sous-unités niveau 2 de la sous-unité active
            sus2 = list(df_su1.sort_values(by='id_sous_unite_N2').Title_su2.dropna().unique())
            
            # Parcourir la liste Sous-unités N2 s'elle n'est pas vide
            if len(sus2) > 0 :
                for su2 in sus2: 
                    
                    # Créer 1 dataframe pour chaque sous-unité
                    df_toprint = df_su1.groupby('Title_su2').get_group(su2).reset_index(drop=True) 
                    
                    # Ajouter un titre 
                    doc.add_heading(su2, 3)
                    
                    # Ajouter les photos + commentaires de la sous-unités niveau 2
                    try :
                        write_table(df_toprint, nbcol)
                        write_no_photo(df_toprint)
                    except : 
                        pass
            
            # Si la liste Sous-unités N2 est vide (N1 <-- dernier niveau)
            else:
                
                # Ajouter les photos + commentaires de la sous-unité N1
                try :
                    write_table(df_su1, nbcol)
                    write_no_photo(df_su1)
                except : 
                    pass
    
    # Si la liste Sous-unités N1 est vide (Unité <-- dernier niveau)
    else: 
        
        # Ajouter les photos + commentaires de l'unité
        try :
            write_table(df_us, nbcol)
            write_no_photo(df_us)
        except : 
            pass
        
# Exporter le fichier doc au même endroit
doc.save(base_url.format(export.format(str(c))))
c = c+1

# Brouillon

In [ ]:
df_merged[(df_merged.IDvisite == id_visite) & (df_merged.Id_su2 == 3.0)]

In [ ]:
doc = docx.Document(base_url.format(template))

paragraph = doc.add_paragraph()
run = paragraph.add_run()
img = Image.open(df_merged.PJ[3][0]['Contenu'])
ext = img.format
img = ImageOps.contain(img, (300, 300)).convert('RGB')
byteIO = io.BytesIO()
img.save(byteIO, format=ext)
run.add_picture(byteIO)

# Exporter le fichier doc au même endroit
doc.save(base_url.format(export.format(str(c))))
c = c+1

In [ ]:
#######################################################################
###########                  ANCIENS CODES                  ###########
#######################################################################

In [ ]:
## (Modèle V2 - Commentaire) 
## Fonction write_each avec l'ancien code tableau adjacent

def write_each(df): 
    # Filtrer seulement la visite sélectionnée
    df = df[df.IDvisite == id_visite]
    
    for i in range(len(df)):
        
        #color = '6699CC'
        color = colors.get(df.Etat[i])
        shading_elm_1 = parse_xml(r"""<w:shd nsdecls('w') w:fill="{}"/>""".format(color)) # format(nsdecls('w'), color)

        ## TEXT ##
        table = doc.add_table(rows=1, cols=4, style="Table Grid")

        # ID du commentaire
        cell_id = table.rows[0].cells
        cell_id[0].text = str(df_merged.Id[i])
        cell_id[0]._tc.get_or_add_tcPr().append(shading_elm_1)
        cell_id[1].merge(cell_id[3])
        cell_id[1].text = ' '

        # Localisation
        cell_loc = table.add_row().cells
        cell_loc[0].text = 'Localisation :' 
        cell_loc[1].merge(cell_loc[3])
        if df_merged.dernier_niveau_u[i]:
            cell_loc[1].text = df_merged.Title_u[i]
        else : 
            if df_merged.dernier_niveau_su1[i]:   
                cell_loc[1].text = df_merged.Title_su1[i]
            else:
                cell_loc[1].text = str(df_merged.Title_su1[i]) + ' - ' + str(df_merged.Title_su2[i])

        # Titre + Commentaire
        cell_com = table.add_row().cells
        cell_com[0].text = 'Commentaire :'
        cell_com[1].merge(cell_com[3])
        if df_merged.Commentaire[i] is None :
            cell_com[1].text = df_merged.Title[i] 
        else : 
            cell_com[1].text = df_merged.Title[i] + ' : ' + df_merged.Commentaire[i]


        # Date de détection
        cell_date = table.add_row().cells
        cell_date[0].text = 'Détecté le :'
        cell_date[1].merge(cell_date[3])
        cell_date[1].text = pd.to_datetime(df_merged.date_detection[i]).strftime("%d/%m/%Y")


        # Etat + Action
        cell_etact = table.add_row().cells
        cell_etact[0].text = 'Etat :'
        cell_etact[1].text = df_merged.Etat[i]
        cell_etact[2].text = 'Action à mener :'
        if df_merged.Action[i] is None :
            cell_etact[3].text = ' '
        else : 
            if df_merged.Duree[i] is None :
                cell_etact[3].text = df_merged.Action[i]
            else : 
                cell_etact[3].text = df_merged.Action[i] + ' - ' + df_merged.Duree[i]

         # Criticité + Durée
        cell_crit = table.add_row().cells
        cell_crit[0].text = 'Criticité :' 
        if df_merged.Criticite[i] is None :
            cell_crit[1].text = ' '
        else :
            cell_crit[1].text = df_merged.Criticite[i]
        cell_crit[2].text = 'Domaine :'
        if df_merged.Domaine[i] is None :
            cell_crit[3].text = ' '
        else :
            cell_crit[3].text = df_merged.Domaine[i]

        # Mettre en forme la police
        for column in table.columns :
            for cell in column.cells : 
                for paragraph in cell.paragraphs :
                    if len(paragraph.runs) > 0:
                        text_style_in_table(cell)    

        # ID du commentaire
        cell_id[0].paragraphs[0].runs[0].font.color.rgb = RGBColor.from_string('FFFFFF')
        cell_id[0].paragraphs[0].runs[0].font.size = Pt(10)
        cell_id[0].paragraphs[0].paragraph_format.alignment = WD_ALIGN_PARAGRAPH.CENTER

        ## PHOTOS ##
        # définir le nombre de colonnes du tableau photos
        nbcol = 3
        if len(df_merged.PJ[i]) > 0:
            # ajouter une ligne au tableau précédent
            cell_photo = table.add_row().cells

            # fusionner toutes les colonnes
            cell_photo[0].merge(cell_photo[3])

            # déterminer le nombre de lignes du tableau photos
            if len(df_merged.PJ[i]) % nbcol == 0:
                cntrows = len(df_merged.PJ[i]) // nbcol
            else:
                cntrows = len(df_merged.PJ[i]) // nbcol + 1

            # ajouter le tableau photos dans la cellule fusionnée
            tbl_photo = cell_photo[0].add_table(rows=1, cols=3) 
            tbl_photo.allow_autofit = True 

            # parcourir chaque ligne et colonne du tableau photos 
            # ajouter les photos au fur à mesure
            for row in range(cntrows):
                for col in range(nbcol):
                    paragraph = tbl_photo.rows[row].cells[col].paragraphs[0]
                    run = paragraph.add_run()
                    try : 
                        img = Image.open(df_merged.PJ[i][row*nbcol+col]['Contenu'])
                        # selon l'orientation de la photo, ajuster la taille 
                        if img.width > img.height:
                            run.add_picture(df_merged.PJ[i][row*nbcol+col]['Contenu'],
                                        width=Cm(5))
                        else:
                            run.add_picture(df_merged.PJ[i][row*nbcol+col]['Contenu'],
                                        height=Cm(4))
                    except :
                        pass  


        """
        # Creer un nouveau tableau adjacent 

        if len(df_merged.PJ[i]) % nbcol == 0:
            cntrows = len(df_merged.PJ[i]) // nbcol
        else:
            cntrows = len(df_merged.PJ[i]) // nbcol + 1

        table1 = doc.add_table(rows=1, cols=3, style="Table Grid")
        table1.allow_autofit = True 

        for row in range(cntrows):
            for col in range(nbcol):
                paragraph = table1.rows[row].cells[col].paragraphs[0]
                run = paragraph.add_run()
                try : 
                    img = Image.open(df_merged.PJ[i][row*nbcol+col]['Contenu'])
                    if img.width > img.height:
                        run.add_picture(df_merged.PJ[i][row*nbcol+col]['Contenu'],
                                    width=Cm(5))
                    # Fixer la longeur si la photo est en mode portrait
                    else:
                        run.add_picture(df_merged.PJ[i][row*nbcol+col]['Contenu'],
                                    height=Cm(4))
                except :
                    pass  
        """

In [ ]:
## (Modèle V2 - Synthèse) 
## Fonction write_in_table sans combinaisions préalables des textes
def write_in_table(dataframe):
    """
    Ecrire un tableau Synthese des observations (sans photo)
    """
    
    # Filtrer seulement la visite sélectionnée
    df = dataframe.copy()
    df = df[df.IDvisite == id_visite]

    # Sélectionner les colonnes nécessaires
    df = df[['Id',     
             'Title_u', 
             'Title', 
             'date_detection', 
             'Etat', 
             'Action', 
             'Duree', 
             'Criticite', 
             'Domaine', 
             'id_sous_unite_N1', 
             'id_sous_unite_N2']].reset_index(drop=True) 
    
    # Remplacer les valeurs None des colonnes textes
    #df[df.select_dtypes('object').columns] = df.select_dtypes('object').fillna(value='')
    
    # Convertir la colonne de date 
    df.date_detection = pd.to_datetime(df.date_detection).dt.strftime("%d/%m/%Y")
    
    # Classer dans l'ordre de la sous unite N1, N2 et l'état
    df = df.sort_values(by=['id_sous_unite_N1', 'id_sous_unite_N2','Etat']).reset_index(drop=True)
    
    # Colonnes de la table 
    syn_col = ["Numéro d’observation", 
               "Localisation", 
               "Commentaire", 
               "Détecté le", 
               "Etat", 
               "Action à mener + durée si complétée", 
               "Criticité + domaine"]
    
    # Initialiser un tableau 
    table = doc.add_table(rows=1, cols=len(syn_col), style="Table Grid")
    # Régler l'alignement
    table.alignment = WD_TABLE_ALIGNMENT.CENTER
    table.allow_autofit = True 
    # Fixer la taille de la 2eme colonne  
    table.columns[2].width = Cm(8) 
    # Fixer la taille de deux dernières colonnes 
    table.columns[len(syn_col)-2].width = Cm(4) 
    table.columns[len(syn_col)-1].width = Cm(4)

    # Créer l'entête
    hdr_cells = table.rows[0].cells
    for i in range(len(syn_col)): 
        hdr_cells[i].text = syn_col[i]

    # Ajouter des données
    for i in range(len(df)):
        # Pour chaque observation, ajouter une ligne
        row_cells = table.add_row().cells
        
        ## Ajouter les données pour les premières colonnes 
        for j in range(len(syn_col)-2): 
            row_cells[j].text = str(df.iloc[i,j])
        
        ## Ajouter Action et durée le cas échéant à l'avant dernière colonne
        if df.loc[i, 'Action'] is None : 
            row_cells[len(syn_col)-2].text = ' '
        else : 
            if df.loc[i, 'Duree'] is not None :
                row_cells[len(syn_col)-2].text = df.loc[i, 'Action'] + ' - ' + df.loc[i, 'Duree']
            else : 
                row_cells[len(syn_col)-2].text = df.loc[i, 'Action']
        
        ## Ajouter Criticité et domaine le cas échéant à la dernière colonne
        if df.loc[i, 'Criticite'] is None : 
            row_cells[len(syn_col)-1].text = ' '
        else:
            if df.loc[i, 'Domaine'] is not None :
                row_cells[len(syn_col)-1].text = df.loc[i, 'Criticite'] + ' - ' + df.loc[i, 'Domaine']
            else : 
                row_cells[len(syn_col)-1].text = df.loc[i, 'Criticite'] 
         
    for column in table.columns :
        for cell in column.cells : 
            for paragraph in cell.paragraphs :
                if len(paragraph.runs) > 0:
                    text_style_in_table(cell)  

In [ ]:
from docx import Document

def make_rows_bold(*rows):
    for row in rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.bold = True

doc = Document()

table = doc.add_table(rows=4, cols=2)


doc.save('test.docx')

In [ ]:
df_merged.columns

In [ ]:
df = df_merged.copy()
df = df[df.IDvisite == id_visite]

# Combiner la localisation de Unité, Sous-unité N1 et N2 dans une nouvelle colonne
df['Localisation'] = (df[['Id','Title_u', 'Title_su1', 'Title_su2']].set_index('Id')
                                                                    .stack()
                                                                    .groupby(level=0, sort=False)
                                                                    .agg(' - '.join)
                                                                    .values)
# Combiner le titre et le commentaire dans une nouvelle colonne
df['Observation'] = (df[['Id','Title', 'Commentaire']].set_index('Id')
                                                      .stack()
                                                      .groupby(level=0, sort=False)
                                                      .agg(': '.join)
                                                      .values)

# Combiner l'action et la durée 
# (!) plus lent sur un grand jeu de données mais accepte les 2 valeurs Null 
df['Action_Duree'] = df[['Action', 'Duree']].apply(lambda x: ' sous '.join(x.dropna()), axis=1)

# Combiner la criticité et le domaine 
# Il faut au moins une valeur notNull
df['Criticite_Domaine'] = (df[['Id','Criticite', 'Domaine']].set_index('Id')
                                                            .stack()
                                                            .groupby(level=0, sort=False)
                                                            .agg(' - '.join)
                                                            .values)

# Sélectionner les colonnes nécessaires
df = df[['Id',     
         'Localisation',
         'Observation',         
         'date_detection', 
         'Etat', 
         'Action_Duree',
         'Criticite_Domaine',
         'id_sous_unite_N1', 
         'id_sous_unite_N2', 'Criticite']].reset_index(drop=True) 

# Convertir la colonne de date 
df.date_detection = pd.to_datetime(df.date_detection).dt.strftime("%d/%m/%Y")

# Classer dans l'ordre de la sous unite N1, N2 et l'état (priorité etat)
df = df.sort_values(by=['Etat', 'id_sous_unite_N1', 'id_sous_unite_N2']).reset_index(drop=True)

# Colonnes de la table 
syn_col = ["Numéro d’observation", 
           "Localisation", 
           "Commentaire", 
           "Détecté le", 
           "Etat", 
           "Action à mener + durée si complétée", 
           "Criticité + domaine"]

doc = Document()

# Initialiser un tableau 
table = doc.add_table(rows=1, cols=len(syn_col), style="Table Grid")
# Régler l'alignement
table.alignment = WD_TABLE_ALIGNMENT.CENTER
table.allow_autofit = True 
# Fixer la taille de la 2eme colonne  
table.columns[2].width = Cm(8) 
# Fixer la taille de deux dernières colonnes 
table.columns[len(syn_col)-2].width = Cm(4) 
table.columns[len(syn_col)-1].width = Cm(4)

# Créer l'entête
hdr_cells = table.rows[0].cells
for i in range(len(syn_col)): 
    hdr_cells[i].text = syn_col[i]

# Ajouter des données
for i in range(len(df)):
    color = colors.get(df.Etat[i])
    color_crit = colors_crit.get(df.Criticite[i])

    # Pour chaque observation, ajouter une ligne
    row_cells = table.add_row().cells

    # Remplir le tableau avec les données du dataframe
    for j in range(len(syn_col)): 
        row_cells[j].text = str(df.iloc[i,j])

# Appliquer le style de la police
for column in table.columns :
    for cell in column.cells : 
        for paragraph in cell.paragraphs :
            if len(paragraph.runs) > 0:
                text_style_in_table(cell)
                


                
doc.save('test.docx')

In [ ]:
def make_rows_bold(*rows):
    for row in rows:
        for cell in row.cells:
            for paragraph in cell.paragraphs:
                for run in paragraph.runs:
                    run.font.bold = True

doc = Document()

table = doc.add_table(rows=4, cols=2)
table.cell(0, 0).text = "Some text"
table.cell(1, 0).text = "Some bold text"
table.cell(1, 1).text = "Some more bold text"
table.cell(2, 0).text = "Some text"
table.cell(3, 1).text = "And more bold text"

make_rows_bold(table.rows[1], table.rows[3])

doc.save('test.docx')